In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

import umap

    


Autosaving every 180 seconds


In [194]:
# plot scatter plots of pixel colors:

#          pup1     pup2    female  male
#clrs= ['red','blue','cyan','green', 'orange']

def load_data(frames_idx,
             features):
    
    x = []
    y = []
    default_mean_nan = 0
    for k in range(n_networks):
        x.append([])
        y.append([])
        for p in features:
            
            # impute x
            temp = tracesx[k*14+p,frames_idx]
            idx = np.where(temp!=0)[0]
            mean = np.nanmean(temp[idx])
            if np.isnan(mean):
                mean = 0
                
            idx1 = np.where(temp==0.)[0]
            temp[idx1]=mean
            idx2 = np.where(np.isnan(temp))[0]
            temp[idx2]=mean            
            x[k].append(temp)
            
            # impute y
            temp = tracesy[k*14+p,frames_idx]
            idx = np.where(temp!=0)[0]
            mean = np.nanmean(temp[idx])
            if np.isnan(mean):
                mean = 0
            idx1 = np.where(temp==0.)[0]
            temp[idx1]=mean
            idx2 = np.where(np.isnan(temp))[0]
            temp[idx2]=mean
            y[k].append(temp)

    x = np.array(x)  
    y = np.array(y)
    print ("x,y:", x.shape, y.shape)

    idx = np.where(np.isnan(x))[0]
    print ("  NANS at assembly time: ", idx)
    
    return x, y


def load_frames_mean_dff(x, y, 
                clrs,
                frames_idx,
                        video_name):
    
    # DOT SIZE
    dot_size = 8
    
    # GRAB FRAM PIXEL INTENSITY
    
    original_vid = cv2.VideoCapture(video_name)
    
    start_frame = frames_idx[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    # get mean
    frames = []
    for n in trange(0, 100, 1):
        ret, frame = original_vid.read()
        frames.append(frame)
    frames = np.array(frames)
    frame_mean = frames.mean(0)   

    vid = np.zeros((x.shape[0],x.shape[1],x.shape[2],3),'int32')
    for n in trange(0, frames_idx.shape[0], 1):
        ret, frame = original_vid.read()

        frame = frame/frame_mean
        
        for k in range(n_networks):
            for p in range(x.shape[1]):
                #print ("y[k,p,n]: ", y[k,p,n])
                tempx = int(y[k,p,n])
                tempy = int(x[k,p,n])
                #print (tempx,tempy)
                vid[k,p,n]=frame[tempx, tempy]
                
    print ("Final vid: ", vid.shape)
    
    idx = np.where(np.isnan(vid))
    print ("VIDEO NANs: ", idx)

    # label final frame for visualization below:
    mark_frame = x.shape[2]//2
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame+mark_frame)
    ret, frame = original_vid.read()
    for a in range(x.shape[0]):
        xx = np.int32(y[a,:,mark_frame])
        yy = np.int32(x[a,:,mark_frame])
 
        for p in range(xx.shape[0]):
            if xx[p]>0:
                frame[xx[p]-dot_size:xx[p]+dot_size,yy[p]-dot_size:yy[p]+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(clrs[a]))*255.).astype('uint8')
    
    original_vid.release()

    # return whole video + example frame
    return vid, frame

def load_frames_adaptive_threshold(x, y, 
                clrs,
                frames_idx,
               video_name):
    
    # DOT SIZE
    dot_size = 8
    jitter=3
    
    # GRAB FRAM PIXEL INTENSITY
    #video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
    original_vid = cv2.VideoCapture(video_name)
    
    start_frame = frames_idx[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    vid = np.zeros((x.shape[0],x.shape[1],x.shape[2],3),'int32')
    for n in trange(0, frames_idx.shape[0], 1):
        ret, frame = original_vid.read()

        #frame=np.array(frame.mean(2),'uint8')
        frame=frame[:,:,0]
        
        frame = cv2.medianBlur(frame,5)
        frame = cv2.adaptiveThreshold(frame,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                    cv2.THRESH_BINARY,11,2)

        frame = cv2.medianBlur(frame,15)

        
        for k in range(n_networks):
            for p in range(x.shape[1]):
                #print ("y[k,p,n]: ", y[k,p,n])
                tempx = int(y[k,p,n])
                tempy = int(x[k,p,n])
                
                #vid[k,p,n]=frame[tempx, tempy]
                 #print (tempx,tempy)
                tempe = frame[tempx-jitter:tempx+jitter, 
                                             tempy-jitter:tempy+jitter]
                #print (tempe.shape)
                tempe_mean = tempe.mean()
                #print (tempe)
                if tempe.shape[0]>0:
                    #print (tempe, np.nanmean(tempe))
                    vid[k,p,n]=np.nanmean(tempe)
                    
                
                
                
    print ("Final vid: ", vid.shape)
    
    idx = np.where(np.isnan(vid))
    print ("VIDEO NANs: ", idx)

    # label final frame for visualization below:
    mark_frame = x.shape[2]//2
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame+mark_frame)
    ret, frame = original_vid.read()
    for a in range(x.shape[0]):
        xx = np.int32(y[a,:,mark_frame])
        yy = np.int32(x[a,:,mark_frame])
 
        for p in range(xx.shape[0]):
            if xx[p]>0:
                frame[xx[p]-dot_size:xx[p]+dot_size,yy[p]-dot_size:yy[p]+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(clrs[a]))*255.).astype('uint8')
    
    original_vid.release()

    # return whole video + example frame
#     print("VID DONE: ", vid.shape)
#     vid = np.array((vid,vid,vid))
#     print (vid.shape)
#     vid = vid.transpose(1,2,3,0)

    return vid, frame


def load_frames(x, y, 
                clrs,
                frames_idx,
               video_name):
    
    # DOT SIZE
    dot_size = 8
    jitter=2
    
    # GRAB FRAM PIXEL INTENSITY
    #video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
    original_vid = cv2.VideoCapture(video_name)
    
    start_frame = frames_idx[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    vid = np.zeros((x.shape[0],x.shape[1],x.shape[2],3),'int32')
    for n in trange(0, frames_idx.shape[0], 1):
        ret, frame = original_vid.read()

        for k in range(n_networks):
            for p in range(x.shape[1]):
                #print ("y[k,p,n]: ", y[k,p,n])
                tempx = int(y[k,p,n])
                tempy = int(x[k,p,n])
                #print (tempx,tempy)
                #vid[k,p,n]=frame[tempx, tempy]
                
                #print (tempx,tempy)
                tempe = frame[tempx-jitter:tempx+jitter, 
                                             tempy-jitter:tempy+jitter]
                #print (tempe.shape)
                tempe_mean = tempe.mean()
                #print (tempe)
                if tempe.shape[0]>0:
                    #print (tempe, np.nanmean(tempe))
                    vid[k,p,n]=np.nanmean(np.nanmean(tempe,0), 0)
                    
                    
                
    print ("Final vid: ", vid.shape)
    
    idx = np.where(np.isnan(vid))
    print ("VIDEO NANs: ", idx)

    # label final frame for visualization below:
    mark_frame = x.shape[2]//2
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame+mark_frame)
    ret, frame = original_vid.read()
    for a in range(x.shape[0]):
        xx = np.int32(y[a,:,mark_frame])
        yy = np.int32(x[a,:,mark_frame])
 
        for p in range(xx.shape[0]):
            if xx[p]>0:
                frame[xx[p]-dot_size:xx[p]+dot_size,yy[p]-dot_size:yy[p]+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(clrs[a]))*255.).astype('uint8')
    
    original_vid.release()

    # return whole video + example frame
    return vid, frame


def load_frames_sobel(x, y, 
                            clrs,
                            frames_idx,
                     video_name):
    
    # DOT SIZE
    dot_size = 8
    
    # GRAB FRAM PIXEL INTENSITY
    #video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
    original_vid = cv2.VideoCapture(video_name)
    
    start_frame = frames_idx[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    jitter=2
    vid = np.zeros((x.shape[0],x.shape[1],x.shape[2]),'float32')
    for n in trange(0, frames_idx.shape[0], 1):
        ret, frame = original_vid.read()
        
        #
        #image = frame.mean(2)
        image = frame[:,:,2]
        
        edge_sobel = filters.sobel(image)
        for k in range(n_networks):
            for p in range(x.shape[1]):
                #print ("y[k,p,n]: ", y[k,p,n])
                tempx = int(y[k,p,n])
                tempy = int(x[k,p,n])
                #print (tempx,tempy)
                tempe = edge_sobel[tempx-jitter:tempx+jitter, 
                                             tempy-jitter:tempy+jitter]
                #print (tempe)
                if tempe.shape[0]>0:
                    #print (tempe, np.nanmean(tempe))
                    vid[k,p,n]=np.nanmean(tempe)
                
    print ("Final vid: ", vid.shape)
    
    idx = np.where(np.isnan(vid))
    print ("VIDEO NANs: ", idx)

    # label final frame for visualization below:
    mark_frame = x.shape[2]//2
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame+mark_frame)
    ret, frame = original_vid.read()
    for a in range(x.shape[0]):
        xx = np.int32(y[a,:,mark_frame])
        yy = np.int32(x[a,:,mark_frame])
 
        for p in range(xx.shape[0]):
            if xx[p]>0:
                frame[xx[p]-dot_size:xx[p]+dot_size,yy[p]-dot_size:yy[p]+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(clrs[a]))*255.).astype('uint8')
    
    original_vid.release()

    # return whole video + example frame
    vid = np.array((vid,vid,vid)).transpose(1,2,3,0)

    return vid, frame



def make_pca_data(vid):

    vid_imputed = vid.copy()
    n_networks = vid_imputed.shape[0]
    print ("# animals ", n_networks)
    
    # IMPUTE MISSNIG DATA BY REPLACING WITH MEAN
    # loop over animal ids
    for id_ in range(n_networks):
        # loop over RGB camera colors
        for k in range(3):
            # loop over features
            for p in range(vid_imputed.shape[1]):
                temp1 = vid_imputed[id_,p,:,k]
                
                #print (temp1)
               
                idx2 = np.where(temp1==0.)[0]
                #print (id_, k, p, "Idx2: ", idx2)

                mean = np.nanmean(temp1)

                idx1 = np.where(np.isnan(temp1)==True)[0]
                temp1[idx1] = mean

                idx = np.where(temp1==0)[0]
                vid_imputed[id_,p,idx,k] = mean

    print ("vid iputed: ", vid_imputed.shape)
    
    vid_pca = vid_imputed.transpose(2,0,1,3)
    #print (vid_pca.shape)

    vid_pca2 = []
    for k in range(n_networks):
        temp = vid_pca[:,k].reshape(vid_imputed.shape[2],-1)
        #print (temp.shape)
        vid_pca2.append(temp)

    vid_pca = np.vstack(vid_pca2)
    print ("vid_pca: ", vid_pca.shape)

    return vid_pca

def make_pca_object(vid_pca):

    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    pca.fit(vid_pca)
    
    return pca

def run_pca(pca, vid_pca,
            vid, 
            title, 
            clrs):

    print ("vid: ", vid.shape)
    pca_2d = pca.transform(vid_pca)

    # plot results

    for k in range(0,vid_pca.shape[0],vid.shape[2]):
        plt.scatter(pca_2d[k:k+vid.shape[2],0],
                   pca_2d[k:k+vid.shape[2],1],
                   c=clrs[k//vid.shape[2]])

def run_umap(umap_obj, 
             vid_pca,
             vid,
             title,
             clrs):
    
    #from sklearn.datasets import fetch_openml
    #import matplotlib.pyplot as plt
    #import seaborn as sns

    #sns.set(context="paper", style="white")

    #mnist = fetch_openml("mnist_784", version=1)
    #print (mnist.data.shape)
    
    embedding = reducer.transform(vid_pca)

    print ("Embeding: ", embedding.shape)
    #fig, ax = plt.subplots(figsize=(12, 10))
    #color = mnist.target.astype(int)

    for k in range(0,embedding.shape[0],vid.shape[2]):
        plt.scatter(embedding[k:k+vid.shape[2],0],
                   embedding[k:k+vid.shape[2],1],
                   c=clrs[k//vid.shape[2]],
                   alpha=.5)

        
def remove_outliers(vid):
    print ("VID: ", vid.shape)
    for a in range(vid.shape[0]):
        for f in range(vid.shape[1]):
            for c in range(vid.shape[3]):
                tempx = vid[a,f,:,c]
                
                idx = np.where(np.isnan(tempx))[0]
                mean = np.nanmean(tempx)
                tempx[idx]=mean
                std = np.std(tempx)*1.1
                idx = np.where(np.logical_or(
                                    tempx>mean+std,
                                    tempx<mean-std))[0]
                tempx[idx]=mean
                
                vid[a,f,:,c]=tempx
    
    return vid



In [42]:
# import umap
# from sklearn.datasets import fetch_openml
# #import matplotlib.pyplot as plt
# #import seaborn as sns

# #sns.set(context="paper", style="white")

# mnist = fetch_openml("mnist_784", version=1)
# print (mnist.data.shape)
# reducer = umap.UMAP(random_state=42)
# embedding = reducer.fit_transform(mnist.data)

In [90]:
# LOAD RAW PCA DATA
if False:
    data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
    vid_start = 0
    locs = [[0, 215], [2180,2468], [4400, 4500],
           [5376,5583]]

    video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'

else:
    video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressed_corrected_nolabels.mp4'
    vid_start = 41050
    data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
    locs = [ 
    [230,500],
        [600,750],
        [800,1200],
        [1400,1800]
        
        
    ]
    
# assign data
tracesx = data['tracesx']
tracesy = data['tracesy']
print (tracesx.shape)
n_networks = tracesx.shape[0]//14
print ("n animals: ", n_networks)


# DEFINE CORRECT ASSEMBLED PERIODS




(70, 2000)
n animals:  5


In [66]:
# def reject_outliers(data, m = 2.):
#     d = np.abs(data - np.median(data))
#     mdev = np.median(d)
#     s = d/mdev if mdev else 0.
#     return data[s<m]

In [195]:
################################################################
################################################################
################ PCA ON SINGLE PIXELS ##########################
################################################################
################################################################

clrs= ['cyan','green', 'red', 'blue', 'pink']
locations = np.arange(4)

#features = np.arange(5,9,1)
features = np.arange(0,10,1)

fig = plt.figure()
pca_data = []
for ctr, location in enumerate(locations):
    ax=plt.subplot(2,2,ctr+1)
    # select x,y from specific chunk of data 
    frames_idx = np.arange(locs[location][0],locs[location][1],1)
    
    x,y = load_data(frames_idx,
                   features)
    
    # grab frames from video
    #vid, frame = load_frames_mean_dff(x, y, 
    vid, frame = load_frames_adaptive_threshold(x, y, 
                             clrs, 
                             frames_idx,
                                     video_name)

    # remove outliars
    vid = remove_outliers(vid)

    # make High Dimensional data;
    vid_pca = make_pca_data(vid)
    pca_data.append(vid_pca)
    # run through fit step
    
    title = str(locs[location])
    plt.title("frames: "+title)
    if True:
        #if False:
        if location == 0:
            pca = make_pca_object(vid_pca)
        run_pca(pca, 
                vid_pca, 
                vid,
                title,
                clrs)
        #plt.xlim(-300,400)
        #plt.ylim(-200,400)
        #plt.xlim(-60,60)
        #plt.ylim(-30,30)
    else:
        #if False:
        if location ==0:
            reducer = umap.UMAP(random_state=42)
            umap_obj = reducer.fit(vid_pca)

    
        umap_data = run_umap(umap_obj,
                             vid_pca,
                             vid,
                             title,
                             clrs)
        plt.xlim(-10,20)
        plt.ylim(-10,20)
    # also show last frame of stack
    #fig=plt.figure()
    #plt.imshow(frame)

plt.suptitle("PCA distributions for 4 (non-id swapping) video segments for "+str(features.shape[0])+ " features"+
             "\n Features = pixel intensity at inferred location", fontsize=20)
#plt.suptitle("UMAP distributions for 4 (non-id swapping) video segments for "+str(features.shape[0])+ " features"+
#             "\n Features = pixel intensity at inferred location", fontsize=20)
plt.show()



<ipython-input-194-41dc8496e074>:20: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])
<ipython-input-194-41dc8496e074>:33: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])


x,y: (5, 10, 270) (5, 10, 270)
  NANS at assembly time:  []


  0%|          | 0/270 [00:00<?, ?it/s]<ipython-input-194-41dc8496e074>:153: RuntimeWarning: Mean of empty slice.
  tempe_mean = tempe.mean()
/home/cat/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 270/270 [00:09<00:00, 29.35it/s]
<ipython-input-194-41dc8496e074>:20: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])
<ipython-input-194-41dc8496e074>:33: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])


Final vid:  (5, 10, 270, 3)
VIDEO NANs:  (array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
VID:  (5, 10, 270, 3)
# animals  5
vid iputed:  (5, 10, 270, 3)
vid_pca:  (1350, 30)
vid:  (5, 10, 270, 3)
x,y: (5, 10, 150) (5, 10, 150)
  NANS at assembly time:  []


  0%|          | 0/150 [00:00<?, ?it/s]<ipython-input-194-41dc8496e074>:153: RuntimeWarning: Mean of empty slice.
  tempe_mean = tempe.mean()
/home/cat/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 150/150 [00:05<00:00, 28.97it/s]


...

x,y: (5, 10, 400) (5, 10, 400)
  NANS at assembly time:  []


100%|██████████| 400/400 [00:13<00:00, 29.31it/s]


Final vid:  (5, 10, 400, 3)
VIDEO NANs:  (array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
VID:  (5, 10, 400, 3)
# animals  5
vid iputed:  (5, 10, 400, 3)
vid_pca:  (2000, 30)
vid:  (5, 10, 400, 3)
x,y: (5, 10, 400) (5, 10, 400)
  NANS at assembly time:  []


<ipython-input-194-41dc8496e074>:20: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])
<ipython-input-194-41dc8496e074>:33: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])
  0%|          | 0/400 [00:00<?, ?it/s]<ipython-input-194-41dc8496e074>:153: RuntimeWarning: Mean of empty slice.
  tempe_mean = tempe.mean()
/home/cat/.local/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 400/400 [00:13<00:00, 29.34it/s]


Final vid:  (5, 10, 400, 3)
VIDEO NANs:  (array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
VID:  (5, 10, 400, 3)
# animals  5
vid iputed:  (5, 10, 400, 3)
vid_pca:  (2000, 30)
vid:  (5, 10, 400, 3)


In [10]:
print (vid.shape)

(5, 14, 207, 3)


In [67]:
################################################################
################################################################
################ PCA ON PATCHES + EDGES ########################
################################################################
################################################################

clrs= ['cyan','green', 'red', 'blue', 'pink']
locations = np.arange(4)
#locations = [0]
features = np.arange(5,6,1)
features = np.arange(0,14,1)

fig = plt.figure()
pca_data = []
for ctr, location in enumerate(locations):
    ax=plt.subplot(2,2,ctr+1)
    # select x,y from specific chunk of data 
    frames_idx = np.arange(locs[location][0],locs[location][1],1)
    
    x,y = load_data(frames_idx,
                   features)
    
    # grab frames from video
    fname_out = '/home/cat/'+str(ctr)+"_sobel.npy"
    if os.path.exists(fname_out)==False:
        vid, frame = load_frames_sobel(x, y, 
                             clrs, 
                             frames_idx)
        np.save(fname_out, vid)
    else:
        vid = np.load(fname_out)
    print ("VID: ", vid.shape)
    
    ## remove outliars
    vid = remove_outliers(vid)

    # make High Dimensional data;
    vid_pca = make_pca_data(vid)
    pca_data.append(vid_pca)
    # run through fit step
    
    title = str(locs[location])
    plt.title("frames: "+title)
    if True:
        #if False:
        if location == 0:
            pca = make_pca_object(vid_pca)
        run_pca(pca, 
                vid_pca, 
                vid,
                title,
                clrs)
        #plt.xlim(-100,100)
        #plt.ylim(-100,100)
        #plt.xlim(-60,60)
        #plt.ylim(-30,30)
    else:
        #if False:
        if location ==0:
            reducer = umap.UMAP(random_state=42)
            umap_obj = reducer.fit(vid_pca)

    
        umap_data = run_umap(umap_obj,
                             vid_pca,
                             vid,
                             title,
                             clrs)
        plt.xlim(-10,20)
        plt.ylim(-10,20)
    # also show last frame of stack
    #fig=plt.figure()
    #plt.imshow(frame)

plt.suptitle("PCA distributions for 4 (non-id swapping) video segments for "+str(features.shape[0])+ " features"+
             "\n Features = pixel intensity at inferred location", fontsize=20)
#plt.suptitle("UMAP distributions for 4 (non-id swapping) video segments for "+str(features.shape[0])+ " features"+
#             "\n Features = pixel intensity at inferred location", fontsize=20)
plt.show()


x,y: (5, 14, 270) (5, 14, 270)
  NANS at assembly time:  []
VID:  (5, 14, 215, 3)
VID:  (5, 14, 215, 3)
# animals  5
vid iputed:  (5, 14, 215, 3)
vid_pca:  (1075, 42)
vid:  (5, 14, 215, 3)


<ipython-input-62-c8941c2541b9>:20: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])
<ipython-input-62-c8941c2541b9>:33: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(temp[idx])


IndexError: list index out of range

In [92]:
pca_data = np.vstack(pca_data)
print (pca_data.shape)

pca = make_pca_object(pca_data)


(4050, 42)


In [81]:
reducer = umap.UMAP(random_state=42)
umap_obj = reducer.fit(pca_data)

/home/cat/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [451]:
# make vid_pca data
vid_pca = make_pca_data(vid)

# make pca object
#pca = make_pca_object(vid_pca)

# 
clrs= ['cyan','green', 'red', 'blue', 'cyan']

title = str(locs[location])
run_pca(pca, 
        vid_pca, 
        vid,
        title,
        clrs)



# animals  5
vid iputed:  (5, 10, 215, 3)
vid_pca:  (1075, 30)
vid:  (5, 10, 215, 3)


In [106]:
from skimage import filters
from skimage.data import camera
from skimage.util import compare_images

#
video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

start_frame = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

vid = np.zeros((x.shape[0],x.shape[1],x.shape[2],3),'int32')
#for n in trange(0, frames_idx.shape[0], 1):
ret, image = original_vid.read()

ax=plt.subplot(2,2,1)
plt.imshow(image)

ax=plt.subplot(2,2,2)
image = np.mean(image,2)
edge_roberts = filters.roberts(image)
plt.imshow(edge_roberts)

ax=plt.subplot(2,2,3)
edge_sobel = filters.sobel(image)

plt.imshow(edge_sobel)
plt.show()
        
        

In [198]:
from scipy.ndimage import gaussian_filter
    
video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
#video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressed_corrected.mp4'

original_vid = cv2.VideoCapture(video_name)

start_frame = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

vid = np.zeros((x.shape[0],x.shape[1],x.shape[2],3),'int32')
#for n in trange(0, frames_idx.shape[0], 1):
frames = []
frames_blured= []
frame_idx = np.random.randint(0,25000,100)
#for n in trange(0, 100, 1):
for n in tqdm(frame_idx):
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, n)

    ret, frame = original_vid.read()
    frames.append(frame)
    
    img = frame[:,:,0]
    #temp = gaussian_filter(temp, sigma=5)
    
    frames_blured.append(temp)
    
# mean subtracted
ax=plt.subplot(2,2,1)
frames = np.array(frames)
print (frames.shape)
frame_means = frames.mean(0)  
print ("frame mean: ", frame_mean.shape)

plt.imshow(frame[:,:,0]-frame_mean[:,:,0])
plt.title("Mean subtracted")

ax=plt.subplot(2,2,2)
img = frame[:,:,0]
img = cv2.medianBlur(img,11)

th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)

plt.imshow(th2)
ax=plt.subplot(2,2,3)
img = frame[:,:,0]
img = cv2.medianBlur(img,15)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

plt.imshow(th3)

# 
ax=plt.subplot(2,2,4)
edge_sobel = filters.sobel(frame[:,:,0])

plt.imshow(edge_sobel)
plt.show()
        

(100, 1024, 1280, 3)
frame mean:  (1024, 1280, 3)


In [24]:
temp = np.array((vid,vid,vid)).transpose(1,2,3,0)
print (temp.shape)
np.save('/home/cat/vid_sobel.npy',temp)

(5, 14, 215, 3)


In [131]:
from skimage import data
from skimage.filters import threshold_otsu, threshold_local

skimage.filters()

image = data.page()

global_thresh = threshold_otsu(image)
binary_global = image > global_thresh

block_size = 25
binary_adaptive = threshold_local(image, block_size, offset=10)

fig, axes = plt.subplots(nrows=3, figsize=(7, 8))
ax0, ax1, ax2 = axes
plt.gray()

ax0.imshow(image)
ax0.set_title('Image')

ax1.imshow(binary_global)
ax1.set_title('Global thresholding')

ax2.imshow(binary_adaptive)
ax2.set_title('Adaptive thresholding')

for ax in axes:
    ax.axis('off')

plt.show()

In [197]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

#img = cv2.imread('dave.jpg',0)
#img = image

original_vid = cv2.VideoCapture(video_name)

start_frame = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

ret, img = original_vid.read()
print (img.shape)
print (type(img[0,0,0]))
img=np.array(img.mean(2),'uint8')
print (img.shape)

ret,th1 = cv2.threshold(img,147,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)

#th2 = cv2.medianBlur(th2,15)


th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
#th3 = cv2.medianBlur(th3,15)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

(1024, 1280, 3)
<class 'numpy.uint8'>
(1024, 1280)
